In [1]:
import json
import logging
import os
import pandas as pd
import random
import sys
from datetime import date
from dotenv import load_dotenv
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from utils.utils import GPT

In [2]:
# params
st_mdl = "gpt-3.5-turbo-1106" # model string
# st_mdl = "gpt-4-1106-preview"
st_task = "multispanqa_logprobs" # name of task

# logging
## today's date
st_today = date.today().strftime("%Y_%m_%d")

## file name:
st_fn = f"{st_today}_{st_task}_n100_{st_mdl}"

## Configure the logger
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

## Create a file handler and set the formatter
file_handler = logging.FileHandler(f"results/{st_fn}.log")
file_handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
file_handler.setFormatter(formatter)

## Create a logger instance and add the file handler
logger = logging.getLogger(__name__)
logger.addHandler(file_handler)

In [3]:
# load api key from .env and create GPT object
load_dotenv()  # Load environment variables from .env file
openai_api_key = os.getenv("OPENAI_API_KEY")
gpt = GPT(openai_api_key, logger=logger, model=st_mdl)

# Load hotpotqa_test_fullwiki.json into a dictionary
with open('data/train.json', 'r') as f:
    data = json.load(f) # keys in each row: ['_id', 'answer', 'question', 'supporting_facts', 'context', 'type', 'level']

In [4]:
def compare_strings(str1, str2):
    """
    This function compares two strings after converting them to lowercase and stripping leading and trailing spaces.
    """
    return str1.lower().strip() == str2.lower().strip()

def check_bool(str1, bool2):
    """
        Checks if string str1 equal to bool bool2
    """
    try:
        return bool(str1) == bool2
    except:
        return False

def compare_str_lst(str1, lst2):
    """
    This function compares a string with each element in a list. If any are equal then returns True
    """
    has_match = [str1.lower().strip() == e.lower().strip() for e in lst2]
    return any(has_match)

In [5]:
# dataset-specific adjustments (we eventually want a list of dicts of format [{"id": "xx", "question": "xxx", "context": "xxx"}, ...]
data = data['data']

# pre-process data
for d in data:
    num_span = d['num_span']
    ctxt = d['context']
    span = ""
    ans = []
    # dataset is in IOB tagging
    for iob_idx, iob in enumerate(d['label']):
        if iob == 'B':
            span = ctxt[iob_idx]
            ans.append(span)
        elif iob == 'I':
            span += ' ' + ctxt[iob_idx]
            ans[-1] = span
        elif iob == 'O' and len(ans) >= num_span:
            break

    d['answer'] = ans
    d['context'] = [["context", [' '.join(ctxt)]]]
    d['question'] = ' '.join(d['question'])


df = pd.json_normalize(data)
df.head()

,id,type,question,context,num_span,label,structure,answer
0,aotqpbqfaswrd0kceud9,HUM,who dies in the shooting in grey's anatomy,"[[context, [`` Death and All His Friends '' is...",2,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",Conjunction,"[Dr. Reed Adamson, Dr. Charles Percy]"
1,nv4tp4lhsa7p6tisaobk,HUM,who wrote i can't make u love me,"[[context, [`` I Ca n't Make You Love Me '' is...",2,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, B, ...",Conjunction,"[Mike Reid, Allen Shamblin]"
2,qfoqf30f0vin5zk9igmz,NUM,when does the north american butterfly associa...,"[[context, [Examples of long - running restric...",2,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",Conjunction,"[July 1, July 4]"
3,lpmaqxlvurem3pf8rgf4,HUM,who made don't hug me i'm scared,"[[context, [Do n't Hug Me I 'm Scared ( often ...",2,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",Conjunction,"[Becky Sloan, Joseph Pelling]"
4,ozwwo1a0yrixj20cdcd1,HUM,who sings all i could do was cry,"[[context, [`` All I Could Do Was Cry '' is a ...",2,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",Non-Redundant,"[Etta James, Beyoncé Knowles]"


## Generate Options

In [14]:
print("Original dataset size:", len(df))
num_questions = 100 # number of questions to run on
indices = random.sample(range(len(data)), num_questions)

item = data[indices[0]]
gold, question, context = item['answer'], item['question'], item['context']
question_with_context = ""
for i in range(len(context)):
    context_str = ''.join(context[i][1])
    question_with_context += f"Paragraph {i+1}: {context[i][0]} {context_str} "
question_with_context += f"\nQuestion: {question}"
logger.info(question_with_context)
res = gpt.get_mc_options(question_with_context, str(gold), sys_prompt_suffix="Each answer should be the string form of a list of values.")
print(res, gold)
print(type(str(gold)))

2024-03-30 16:08:25,626 - INFO - Paragraph 1: context The following article details the history of Monday Night Football , the weekly broadcast of National Football League games on U.S. television . During the early 1960s , NFL Commissioner Pete Rozelle envisioned the possibility of playing at least one game weekly during prime time that could be viewed by a greater television audience ( while the NFL had scheduled Saturday night games on the DuMont Television Network in 1953 and 1954 , poor ratings and the dissolution of DuMont led to those games being eliminated by the time CBS took over the rights in 1956 ) . An early bid by the league in 1964 to play on Friday nights was soundly defeated , with critics charging that such telecasts would damage the attendance at high school football games . Undaunted , Rozelle decided to experiment with the concept of playing on Monday night , scheduling the Green Bay Packers and Detroit Lions for a game on September 28 , 1964 . While the game was n

Original dataset size: 5230


2024-03-30 16:08:27,621 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


ChatCompletion(id='chatcmpl-98ZI11DQOQjh4lMNT8Lz1XKib0L1s', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_REPYLjZN9gtXtTUP9GBrKyhu', function=Function(arguments='{"a":["Green Bay Packers","Detroit Lions"],"b":["Dallas Cowboys","New York Giants"],"c":["New England Patriots","Kansas City Chiefs"],"d":["San Francisco 49ers","Seattle Seahawks"]}', name='return_list'), type='function')]))], created=1711829305, model='gpt-3.5-turbo-1106', object='chat.completion', system_fingerprint='fp_89448ee5dc', usage=CompletionUsage(completion_tokens=40, prompt_tokens=468, total_tokens=508)) ['Green Bay Packers', 'Detroit Lions']
<class 'str'>


## Generator

In [ ]:
predictions = [] # list of predictions, reset after each set of runs
answers = []
indices = [] # question numbers
evals = [] # whether prediction matches answer
questions = [] # list of questions


# Evaluate (use randomly selected indices from option generation step
for idx in indices:
    item = data[idx]
    # if item['level'] == 'hard':
    gold, question, context = item['answer'], item['question'], item['context']
    question_with_context = ""
    for i in range(len(context)):
        context_str = ''.join(context[i][1])
        question_with_context += f"Paragraph {i+1}: {context[i][0]} {context_str} "
    question_with_context += f"Q: {question}"
    logger.info(question_with_context)
    msg = [{"role": "system", "content": "Answer in a single noun, verb, or adjective"}, {"role": "user", "content": question_with_context}]
    predictions.append(gpt.gen(msgs=msg).content)
    answers.append(gold)
    evals.append(compare_str_lst(predictions[-1], gold))
    questions.append(question)
    print(predictions[-1])

# Print the predictions and answers
logger.info(predictions)
logger.info(answers)
percent_correct = (sum(evals) / len(evals)) * 100

print(f"Percentage of True values: {percent_correct}%")

# create a dataframe with the data
df = pd.DataFrame({'Index': indices, 'Question': questions, 'Prediction': predictions, 'Answer': answers, 'Eval': evals})

# save the datafrme to csv
df.to_csv(f'results/{st_fn}.csv', index=False)

In [6]:
item = data[0]
# if item['level'] == 'hard':
gold, question, context = item['answer'], item['question'], item['context']
question_with_context = ""
for i in range(len(context)):
    context_str = ''.join(context[i][1])
    question_with_context += f"Paragraph {i+1}: {context[i][0]} {context_str} "
question_with_context += f"Q: {question}"
logger.info(question_with_context)
msg = [{"role": "system", "content": "Answer in a single noun, verb, or adjective"}, {"role": "user", "content": question_with_context}]
comp = gpt.gen_top20(msgs=msg)

2024-03-29 20:44:01,885 - INFO - Paragraph 1: context `` Death and All His Friends '' is the season finale of the sixth season of the American television medical drama Grey 's Anatomy , and the show 's 126th episode overall . It was written by Shonda Rhimes and directed by Rob Corn . The episode was originally broadcast on the American Broadcasting Company ( ABC ) in the United States on May 20 , 2010 . The episode was the second part of the two - hour season six finale , the first being Sanctuary , and took place at the fictional Seattle Grace Hospital . The original episode broadcast in the United States had an audience of 16.13 million viewers and opened up to universal acclaim . The episode centers a shooting spree at the hospital by a former patient 's husband Gary Clark ( Michael O'Neill ) . The episode marked the last appearances for Nora Zehetner and Robert Baker as Dr. Reed Adamson and Dr. Charles Percy respectively as both the characters were killed in the shooting . In the e

In [7]:
print(comp)

([-0.7024841, -1.3044682, -2.6844387, -3.1696968, -3.5967903, -4.34972, -4.373026, -4.496279, -4.6449203, -4.8700867, -4.9958572, -5.3759212, -5.441187, -5.584915, -5.5931377, -5.649399, -5.8335323, -6.433216, -6.6694927, -6.824173], ['Charles', 'Gary', 'Re', 'character', 'Character', 'characters', 'Dr', 'Doctor', 'Sh', 'P', 'doctor', 'Characters', 'N', 'Clark', 'episode', 'Episode', 'sh', 'shoot', 'spo', 'Spoiler'])


## Validator

### Baseline

In [ ]:
fn_sfx = "" # filename suffix, used when saving validator file
sys_prompt = "Answer 'TRUE' if the answer is correct and 'FALSE' if the answer is incorrect."

### Search for optimal prompt

In [6]:
fn_sfx = "_prmpt" # filename suffix, used when saving validator file
sys_prompt = "Think step by step. First generate your own answer to the question and then compare this with the provided answer. Check, then double check your thinking. The last word of your response should be 'TRUE' if the answer is correct and 'FALSE' if the answer is incorrect, given the question."

### Chain of Thought

In [16]:
fn_sfx = "_cot"
sys_prompt = "Answer 'TRUE' if the answer is correct and 'FALSE' if the answer is incorrect, explaining your reasoning before answering. Think step by step."

### Run validator

In [17]:
# gen_data_fn = f"results/{st_fn}.csv" # change this to the filename of the generator results
gen_data_fn = 'results/2024_02_13_multispanqa_n100_gpt-4-1106-preview.csv'

df = pd.read_csv(gen_data_fn)
indices = df.iloc[:, 0].tolist()
indices = indices[:-1] # just because we manually added an additional row on the CSV when calculating scores

val_preds = [] # list of predictions for validator, reset after each set of runs
val_evals = [] # whether prediction matches answer

# Evaluate
for _, row in df.iterrows():
    idx = row['Index']
    ans = row['Prediction']
    gold = row['Manual Eval']
    
    item = data[idx]
    # if item['level'] == 'hard':
    question, context = item['question'], item['context']
    question_with_context = ""
    for i in range(len(context)):
        context_str = ''.join(context[i][1])
        question_with_context += f"Paragraph {i+1}: {context[i][0]} {context_str} "
    question_with_context += f"Q: {question}"
    question_with_context += f"A: {ans}"
    logger.info(question_with_context)
    msg = [{"role": "system", "content": sys_prompt}, {"role": "user", "content": question_with_context}]
    val_preds.append(gpt.gen(msgs=msg).content)
    val_evals.append(check_bool(val_preds[-1], gold))
    print(val_preds[-1])

# Print the predictions and answers
logger.info(val_preds)
logger.info(df['Manual Eval'].tolist())
percent_correct = (sum(val_evals) / len(val_evals)) * 100

print(f"Percentage of True values: {percent_correct}%")

# add validator predictions to column
df['Validator Pred'] = val_preds

# save the dataframe to csv
df.to_csv(f'results/{st_fn}_with_val{fn_sfx}.csv', index=False);

2024-02-15 20:53:23,176 - INFO - Paragraph 1: context The next election will take place in 2022 . Current President Emmanuel Macron is eligible for re-election in that year . With the exception of senatorial election , for which there is an electoral college , the voters are French citizens over the age of 18 registered on the electoral rolls . People are automatically registered on reaching the age of 18 . For municipal and European , but not national , elections , citizens aged 18 or older of other European Union countries may vote in France . Registration is not compulsory , but the absence of registration precludes the possibility of voting . Citizens may register either in their place of residence or in a place where they have been on the roll of taxpayers for local taxes for at least 5 years , but not in more than one place . Citizens living abroad may register at the consulate responsible for the region in which they live . Q: who has the right to vote in franceA: citizens
2024-

TRUE

The information provided indicates that the voters in France are French citizens over the age of 18 registered on the electoral rolls. Additionally, for municipal and European elections, citizens aged 18 or older of other European Union countries may also vote in France, but the primary premise is that the right to vote is held by citizens of France who meet the age and registration requirements.


2024-02-15 20:53:32,151 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-15 20:53:32,153 - INFO - Paragraph 1: context One Tree Hill is an American television drama series created by Mark Schwahn , which premiered on September 23 , 2003 , on The WB . After the series ' third season , The WB merged with UPN to form The CW , and since September 27 , 2006 , the network has been the official broadcaster of the series in the United States . The show is set in the fictional town of Tree Hill in North Carolina and originally follows the lives of two half - brothers , Lucas Scott ( Chad Michael Murray ) and Nathan Scott ( James Lafferty ) , who compete for positions on their school 's basketball team , and the drama that ensues from the brothers ' romances . Most of the filming took place in and around Wilmington , North Carolina . Many of the scenes were shot near the battleship USS North Carolina and on the University of North Carolina Wilmingt

TRUE

The three tribes that make up the Sioux Nation are indeed the Lakota, Western Dakota, and Eastern Dakota, as mentioned in the paragraph. These tribes are categorized based on regional and linguistic characteristics, forming the larger collective known as the Great Sioux Nation.


2024-02-15 20:53:35,894 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-15 20:53:35,896 - INFO - Paragraph 1: context The song was nominated for ' Best British Single ' at the 2013 BRIT Awards , and was ranked the twelfth best - selling single of 2012 in the UK . `` Beneath Your Beautiful '' is an R&B ballad written by Labrinth , Mike Posner and Emeli Sandé . It was recorded as a duet between Labrinth and Sandé for Labrinth 's debut studio album Electronic Earth . The title of the song attracted attention from fans , some of whom questioned Labrinth about the spelling of the word `` Your '' within the song 's title , believing that was a grammatical error and should have been spelt `` You 're '' ( although the song 's lyrics clearly refer to it as a noun ) . Upon reaching number one , fans asked Labrinth via social networking website Twitter about the spelling . He replied `` Can I make something clear ! this is twitter ! Not an English 

...

TRUE

Aman Gandotra and Natasha Bharadwaj were declared winners of the 2018 season of India's Next Superstars, which indicates that they won the grand finale of the show.


2024-02-15 21:03:13,440 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-15 21:03:13,442 - INFO - Paragraph 1: context Philo Taylor Farnsworth ( August 19 , 1906 -- March 11 , 1971 ) was an American inventor and television pioneer . He made many contributions that were crucial to the early development of all - electronic television . He is perhaps best known for his 1927 invention of the first fully functional all - electronic image pickup device ( video camera tube ) , the `` image dissector '' , as well as the first fully functional and complete all - electronic television system . He was also the first person to demonstrate such a system to the public . Farnsworth developed a television system complete with receiver and camera , which he produced commercially in the form of the Farnsworth Television and Radio Corporation , from 1938 to 1951 , in Fort Wayne , Indiana . In later life , Farnsworth invented a small nuclear fusion device , 

TRUE

The girl locked up in "Don't Breathe" is indeed Cindy. As described in the paragraph, she is the rich young woman held captive by the Blind Man, and it is revealed that she was involved in a car accident for which the Blind Man holds her responsible for the death of his child. This is confirmed when she shows Rocky and Alex a newspaper article about the car accident and they recognize her as Cindy.


2024-02-15 21:03:17,971 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-15 21:03:17,973 - INFO - ['TRUE\n\nThe information provided indicates that the voters in France are French citizens over the age of 18 registered on the electoral rolls. Additionally, for municipal and European elections, citizens aged 18 or older of other European Union countries may also vote in France, but the primary premise is that the right to vote is held by citizens of France who meet the age and registration requirements.', 'TRUE\n\nThe three tribes that make up the Sioux Nation are indeed the Lakota, Western Dakota, and Eastern Dakota, as mentioned in the paragraph. These tribes are categorized based on regional and linguistic characteristics, forming the larger collective known as the Great Sioux Nation.', 'TRUE\n\nThe show "One Tree Hill" was primarily filmed in and around Wilmington, North Carolina, even though the show is set in the fictional town of Tr

TRUE

Philo Taylor Farnsworth is indeed recognized for inventing the first fully functional all-electronic television system, and his key invention, the "image dissector" camera tube, was demonstrated in 1927. As such, it is accurate to say that Philo Farnsworth invented the television, with 1927 being the year he made his significant breakthrough.
Percentage of True values: 69.0%
